In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import torch
import os
import copy
import csv
import time

from agent import PPO
from env import Flowsheet
from Simulation import *

In [ ]:
def plot_learning_curve(x, scores):
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
    plt.plot(x, running_avg)
    plt.plot(x,scores, alpha=0.2)
    plt.title('Running average of previous 100 scores')

In [ ]:
global sim, score_history

def main():
    global sim, score_history
    cwd = os.getcwd()
    sim = Simulation("DME_prod.bkp", cwd)


    env_kwargs = {
            "sim": sim,
            "pure": 0.99, 
            "max_iter": 15, 
            "inlet_specs": [25.0, 1, {"DME": 0, "WATER": 0.2*261.5, "METHANOL": 0.8*261.5}]
        }

    env = Flowsheet(**env_kwargs)

    random_seed = 100
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)
    env.seed(random_seed)
   

    # Hyperparameters
    kwargs = {
        "state_dim": env.observation_space.shape[0], 
        "action_dim": env.action_space.n, 
        "env_with_Dead": True,
        "gamma": 0.99, 
        "gae_lambda": 0.95, 
        "policy_clip": 0.2, 
        "n_epochs": 10, 
        "net_width": 64, 
        "lr": 2.5e-4, 
        "l2_reg": 0.5, 
        "batch_size": 64, 
        "entropy_coef": 0.025,
        "adv_normalization": True, 
        "entropy_coef_decay": 0.9
    }


    N = 512 # lenth of long trajectory
    Max_train_steps = int(50e3)
    best_interval = int(10e3)
    score_history = []

    total_steps = 0
    traj_length = 0
    episode = 1
    best_score = -100
    
    
    model = PPO(**kwargs)

    if not os.path.exists('model'): 
        os.mkdir('model')
    
    if not os.path.exists('best_model'): 
        os.mkdir('best_model')
    
    update = 0
    num_updates = Max_train_steps // N
    
    while total_steps < Max_train_steps:
        (observation, sin), done, steps, score = env.reset(), False, 0, 0
        mask_vec = env.action_masks(sin, True)

        '''Interact & trian'''
        while not done:
            steps += 1
            traj_length += 1
            total_steps += 1

            action, probs = model.select_action(observation, mask_vec)
            # Sometimes ASPEN bugs and fails to read information
            try:
                observation_, reward, done, info, sin = env.step(action, sin)

                if (done and steps != env.max_iter):
                    dw = True  #dw: dead and win
                else:
                    dw = False

                model.put_data((observation, action, reward, observation_, probs, done, dw, mask_vec))
                observation = observation_
                mask_vec = env.action_masks(sin)
                score += reward       

                if traj_length % N == 0:
                    a,b,c = model.train()
                    traj_length = 0
                    update += 1
                    
                    frac = 1.0 - (update - 1.0) / num_updates
                    lrnow = frac * kwargs["lr"]
                    model.actor.optimizer.param_groups[0]["lr"] = lrnow
                    model.critic.optimizer.param_groups[0]["lr"] = lrnow

                    model.save(update)
            
            except:
                print("Error happened when reading data")
                done = True
        
            
        
        
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])

        # Save values of rewards
        f = open('rewards_file', 'a')
        writer = csv.writer(f, lineterminator = '\n')
        writer.writerow([avg_score])
        f.close()

        print('Episode {} total steps {} avg score {:.4f}'.
              format(episode, total_steps, avg_score))
        episode += 1

        ''' best model '''
        if total_steps >= best_interval:
            if score_history[-1] > best_score:
                best_score = score_history[-1]
                model.best_save()


    sim.CloseAspen() 

    
if __name__ == '__main__':
    start_time = time.time()

    try:
        main()
    except:
        print("Error")
        sim.CloseAspen()

    
    x = [i+1 for i in range(len(score_history))]
    plot_learning_curve(x, score_history)

    end_time = time.time()
    final_time = end_time - start_time
    print(final_time)